In [3]:
from src.services.llm import LLM
from src.services.logical_tree import LogicalTree
from src.utils.workflow import solve_goal_list
from typing import Callable, List, Dict, Any
from src.services.agents import Tool_Calls

# from src.services.agents import Agent, Tool
from src.services.custom_tools import custom_tools, clean_training_logs
from src.services.external_client import ExternalClient
import glob
import os

os.environ["NO_PROXY"] = "*"

# llm = LLM(model_name="qwen3:8b")

with open(".webui_port", "r", encoding="utf-8") as f:
    port = int(f.read())
# ec = ExternalClient(port=port)
llm = LLM(
    model_name="deepseek-chat",
    llm_url="https://api.deepseek.com/chat/completions",
    api_key="sk-2332c3d16a8d4f4ba1b3503074ba04c5",
    format="openai",
    # ec=ec,
)
# llm = LLM(
#     model_name="gpt-5-2025-08-07",
#     llm_url="https://api.aimlapi.com/v1/chat/completions",
#     api_key="9ce046a9681446c48427b3fe4dd7cdd4",
#     format="openai",
# )
# llm = LLM(
#     model_name="gpt-4o",
#     llm_url="https://api.aimlapi.com/v1/chat/completions",
#     api_key="9ce046a9681446c48427b3fe4dd7cdd4",
#     format="openai",
# )


ct = custom_tools(
    MAIN_DIR="/data/yuanwen/workspace/gitauto",
    PYTHON_PATH="python",
    MATLAB_PATH="/Applications/MATLAB_R2023b.app/bin/matlab",
    LOCAL_TMP_PATH="/data/yuanwen/workspace/tmp",
    PMC_URL="https://pmc.ncbi.nlm.nih.gov/articles/PMC7567795",
    llm=llm,
)
from src.services.llm import load_messages, save_messages

tool_calls_path = (
    "/data/yuanwen/workspace/tmp/tool_calls_path.json"
)
env_tool_calls_path = "/workspace/tool_calls_path.json"

tc = Tool_Calls(PATH=tool_calls_path, ENV_PATH=env_tool_calls_path, MAX_CHAR=50000)
tc.clear()

tools = [
    ct.func_ls,
    ct.func_cat,
    # ct.func_cat_with_llm,
    ct.func_write,
    ct.func_modify,
    ct.func_insert,
    ct.func_append,
    ct.func_prepend,
    ct.func_python,
    ct.func_cmd,
    ct.func_matlab,
    # ct.func_fetch_info_from_pmc_with_llm,
    ct.func_human,
    ct.func_guide,
    # ct.func_think,
    ct.add_child,
    ct.return_to_parent,
]
s = ct.func_youtube_search_with_transcript("quantum qubit", order="relevance", include_transcript=False)

KeyboardInterrupt: 

In [2]:
s

'🎬 **What is a Qubit? - A Beginner&#39;s Guide to Quantum Computing** (Mr Verse, 2023-01-08T06:54:36Z)\n🔗 https://www.youtube.com/watch?v=90za6mazNps\n📝 Description: What is a qubit? Just as a classical bit has a state – either 0 or 1 – a qubit also has a state. Two possible states for a qubit are the ...\n📜 Transcript (partial): the bid is the most basic unit of...\n----------------------------------------\n\n🎬 **What are qubits?  #physics #iyq2025 #education #quantum #stem  #quantumscience #quantum #qubits** (National Institute of Standards and Technology, 2025-02-24T16:19:27Z)\n🔗 https://www.youtube.com/watch?v=o3eIEMFV2CM\n📝 Description: Step into the fascinating world of quantum mechanics in this quick explainer and find out a little BIT about what makes QUBITS so ...\n📜 Transcript (partial): [No transcript available: \nCould not retrieve a transcript for the video https://www.youtube.com/watch?v=o3eIEMFV2CM! This is most likely caused by:\n\nYouTube is blocking requests from your

In [ ]:
# 最小可用案例：获取 YouTube 视频字幕
from youtube_transcript_api import YouTubeTranscriptApi

def get_partial_transcript(video_id: str, max_segments: int = 5) -> str:
    """
    从视频中获取前 max_segments 段字幕文本拼成一行。
    """
    try:
        api = YouTubeTranscriptApi()
        transcript = api.fetch(video_id, languages=['en'])
        # 只取前几个字幕段
        segments = transcript[:max_segments]
        text = " ".join([seg["text"] for seg in segments])
        return text
    except Exception as e:
        return f"[No transcript available: {e}]"

if __name__ == "__main__":
    VIDEO_ID = "90za6mazNps"  # 替换成你需要的视频 ID
    snippet = get_partial_transcript(VIDEO_ID, max_segments=10)
    print("Transcript snippet:", snippet)

Transcript snippet: the bid is the most basic unit of information in classical Computing and classical information in Quantum computation and Quantum information a Quantum bit or qubit plays a similar role in this video we will see what the qubit actually means and compare its properties to those of classical bits we're going to describe qubits as



=== Prompt step 1 ===

Failed to parse JSON from line: -alive
根据之前的tool calls记录，我找到了一个已经没有用的tool call：

**call_01_ON4j8QJUR3YBW6IeJZzG4GuB**

这个tool call是用来修改文件路径的：
- 函数名：func_modify
- 文件路径：package/test_allpatient.py  
- 修改内容：将`"./Results/Hebeipredicttest.txt"`改为`"/workspace/package/Results/Hebeipredicttest.txt"`

这个修改已经执行过并且成功了，现在这个tool call已经没有用了，因为：
1. 文件修改已经完成
2. 后续的测试运行已经使用了新的路径
3. 预测结果已经成功生成在指定位置

'根据之前的tool calls记录，我找到了一个已经没有用的tool call：\n\n**call_01_ON4j8QJUR3YBW6IeJZzG4GuB**\n\n这个tool call是用来修改文件路径的：\n- 函数名：func_modify\n- 文件路径：package/test_allpatient.py  \n- 修改内容：将`"./Results/Hebeipredicttest.txt"`改为`"/workspace/package/Results/Hebeipredicttest.txt"`\n\n这个修改已经执行过并且成功了，现在这个tool call已经没有用了，因为：\n1. 文件修改已经完成\n2. 后续的测试运行已经使用了新的路径\n3. 预测结果已经成功生成在指定位置'


[LLM] Query
  [SYSTEM PROMPT]
    
  [INPUT]
    测试
  [OUTPUT]
    您好！我很乐意为您提供帮助！😊
    
    看起来您是在测试我的功能，这是完全正常的！让我简单介绍一下我能为您做些什么：
    
    **我能帮您：**
    - 回答各种问题和解答疑惑
    - 协助写作、翻译、编程等任务
    - 分析和处理您上传的文件（图片、PDF、Word、Excel等）
    - 进行创意思考和问题解决
    - 提供学习和工作上的支持
    
    **我的特点：**
    - 完全免费使用
    - 支持128K上下文长度
    - 可以联网搜索（需要您手动开启）
    - 知识更新到2024年7月
    
    请随时告诉我您需要什么帮助，或者想测试什么具体功能。我会尽我所能为您提供最好的服务！✨
    
    有什么具体的问题或任务想要尝试吗？


'您好！我很乐意为您提供帮助！😊\n\n看起来您是在测试我的功能，这是完全正常的！让我简单介绍一下我能为您做些什么：\n\n**我能帮您：**\n- 回答各种问题和解答疑惑\n- 协助写作、翻译、编程等任务\n- 分析和处理您上传的文件（图片、PDF、Word、Excel等）\n- 进行创意思考和问题解决\n- 提供学习和工作上的支持\n\n**我的特点：**\n- 完全免费使用\n- 支持128K上下文长度\n- 可以联网搜索（需要您手动开启）\n- 知识更新到2024年7月\n\n请随时告诉我您需要什么帮助，或者想测试什么具体功能。我会尽我所能为您提供最好的服务！✨\n\n有什么具体的问题或任务想要尝试吗？'